In [4]:
import numpy as np
import pyDOE
import random
import os
import sys
import matplotlib.pyplot as plt
import flopy
import pandas as pd

# 生成初始拉丁超立方设计
def generate_initial_lhs(n, d, bounds):
    lhs = pyDOE.lhs(d, samples=n)
    X_init = np.zeros((n, d))
    for i in range(d):
        X_init[:, i] = bounds[i][0] + (bounds[i][1] - bounds[i][0]) * lhs[:, i]
    return X_init


# 退火参数初始化
def initialize_annealing_parameters():
    T0 = 1000
    alpha = 0.95
    Nmax = 1000
    return T0, alpha, Nmax


# 成本函数，最大化样本间最小距离
def cost_function(X):
    n = X.shape[0]
    min_dist = float('inf')
    for i in range(n):
        for j in range(i + 1, n):
            dist = np.linalg.norm(X[i] - X[j])
            min_dist = min(min_dist, dist)
    return min_dist


# 生成邻域解（改进版）
def generate_neighbor(X, mode='both'):
    n, d = X.shape
    X_candidate = X.copy()

    if mode == 'Q_only':
        # 仅扰动Q列（第0列）
        swap_indices = random.sample(range(n), max(1, n // 10))  # 扰动10%的样本
        np.random.shuffle(X_candidate[swap_indices, 0])

    elif mode == 'R_only':
        # 仅扰动R列（第1列）
        swap_indices = random.sample(range(n), max(1, n // 10))
        np.random.shuffle(X_candidate[swap_indices, 0])

    elif mode == 'both':
        # 同时扰动两个维度
        j = random.choice([0, 1])
        swap_indices = random.sample(range(n), max(1, n // 10))
        X_candidate[swap_indices, j] = np.random.permutation(X_candidate[swap_indices, j])

    return X_candidate


# Metropolis接受准则
def metropolis_acceptance(DeltaC, T):
    if DeltaC > 0:
        return True
    else:
        return random.random() < np.exp(DeltaC / T)


# 终止条件判定
def should_terminate(improvement_rates, T, T_min, Nmax, iteration):
    if len(improvement_rates) >= 10 and all(abs(rate) < 0.01 for rate in improvement_rates[-10:]):
        return True
    if T <= T_min:
        return True
    if iteration >= Nmax:
        return True
    return False


# 模拟退火算法（增加模式参数）
def simulated_annealing(X_init, T0, alpha, Nmax, T_min=1, mode='both'):
    X_current = X_init
    C_current = cost_function(X_current)
    improvement_rates = []

    for iteration in range(Nmax):
        X_candidate = generate_neighbor(X_current, mode=mode)
        C_candidate = cost_function(X_candidate)
        DeltaC = C_candidate - C_current

        if metropolis_acceptance(DeltaC, T0):
            X_current = X_candidate
            C_current = C_candidate

        improvement_rate = (C_candidate - C_current) / C_current if C_current != 0 else 0
        improvement_rates.append(improvement_rate)

        if should_terminate(improvement_rates, T0, T_min, Nmax, iteration):
            break

        T0 = alpha * T0

    return X_current

# 确保目标目录存在
output_dir = "confined_sample"
os.makedirs(output_dir, exist_ok=True)  # 自动创建目录（若不存在）

#定义参数
sim_name = "con"
length_units = "meters"
time_units = "days"

# 文档参数
nper = 30  # 总模拟时间 10 年，分 20 个应力周期
nlay = 1
nrow = 30  # 假设值，需根据实际调整
ncol = 50  # 根据代码中最大列数调整
delr = 100.0
delc = 100.0
top = 100.0  # 假设值，需根据实际调整
botm = np.full((nlay, nrow, ncol), 60.0)  # 所有单元格层底高程为60m
icelltype = 1
strt = np.full((nlay, nrow, ncol), 70.0)  # 所有单元格初始水头70m

# 定水头边界条件
chd_spd = {
    0: [
        [0, row, 6, 100] for row in range(0, 2)
    ] + [
        [0, 2, col , 100] for col in range(4, 7)
    ] + [
        [0, row, 4, 100] for row in range(2, 5)
    ] + [
        [0, 6, col, 100] for col in range(2, 4)
    ] + [
        [0, row , 4, 100] for row in range(4, 7)
    ] + [
        [0, 8, col, 100] for col in range(0, 2)
    ] + [
        [0, row, 2, 100] for row in range(6, 9)
    ] + [
        [0, row, 41, 80] for row in range(28, 30)
    ] + [
        [0, 28, col, 80] for col in range(42, 44)
    ] + [
        [0, row , 43, 80] for row in range(26, 29)
    ] + [
        [0, 26, col, 80] for col in range(44, 46)
    ] + [
        [0, row, 45, 80] for row in range(24, 27)
    ] + [
        [0, 24, col, 80] for col in range(46, 48)
    ] + [
        [0, row, 47, 80] for row in range(22, 25)
    ] + [
        [0, 22, col, 80] for col in range(48, 50)
    ] 
}
unique_chd_spd = {0: []}
used_cells = set()
for item in chd_spd[0]:
    lay, row, col, _ = item
    cell_key = (lay, row, col)
    if cell_key not in used_cells:
        unique_chd_spd[0].append(item)
        used_cells.add(cell_key)
    else:
        print(f"已跳过重复的定水头单元格: ({lay}, {row}, {col})")

chd_spd = unique_chd_spd
# 水头边界和渗透系数分布相关代码
# 初始化 idomain 数组，所有单元格默认为活动状态
idomain = np.ones((nlay, nrow, ncol), dtype=np.int32)

# 原离散区域，修正了缺少逗号的问题
original_regions = [
    (0, 0, *range(0, 6)),
    (0, 2, *range(0, 4)),
    (0, 4, *range(0, 4)),
    (0, 6, *range(0, 2)),
    (0, 8, 0),
    (0, 12, *range(0, 2)),
    (0, 14, *range(0, 2)),
    (0, 16, *range(0, 4)),
    (0, 18, *range(0, 6)),
    (0, 20, *range(0, 8)),
    (0, 22, *range(0, 24)),
    (0, 24, *range(0, 26)),
    (0, 26, *range(0, 32)),
    (0, 28, *range(0, 34)),
    (0, 29, *range(0, 38))
]

# 存储连接后的区域
connected_regions = []

# 遍历原区域列表
for i in range(len(original_regions)):
    if i == 0:
        # 第一行直接添加
        connected_regions.append(original_regions[i])
    else:
        # 获取上一行和当前行的信息
        prev_row = original_regions[i - 1][1]
        current_row = original_regions[i][1]
        prev_cols = set(original_regions[i - 1][2:])
        current_cols = set(original_regions[i][2:])

        # 找出相邻两行列索引的最小和最大值
        min_col = min(min(prev_cols), min(current_cols))
        max_col = max(max(prev_cols), max(current_cols))

        # 填充上一行和当前行之间的空缺行
        for row in range(prev_row + 1, current_row):
            connected_regions.append((0, row, *range(min_col, max_col + 1)))

        # 添加当前行
        connected_regions.append(original_regions[i])

# 标记非活动单元格
for region in connected_regions:
    lay = region[0]
    row = region[1]
    for col in region[2:]:
        idomain[lay, row, col] = -1

# 让 20 到 25 行保持活动状态
for row in range(8, 10):
    idomain[0, row, :2] = 1

original_regions = [
    (0, 0, *range(16, 50)),
    (0, 2, *range(20, 50)),
    (0, 4, *range(30, 50)),
    (0, 6, *range(32, 50)),
    (0, 8, *range(42, 50)),
    (0, 10, *range(46, 50)),
    (0, 12, *range(48, 50)),
    (0, 24, *range(48, 50)),
    (0, 26, *range(46, 50)),
    (0, 28, *range(44, 50)),
    (0, 29, *range(42, 50)),
]

# 存储连接后的区域
connected_regions = []

# 遍历原区域列表
for i in range(len(original_regions)):
    if i == 0:
        # 第一行直接添加
        connected_regions.append(original_regions[i])
    else:
        # 获取上一行和当前行的信息
        prev_row = original_regions[i - 1][1]
        current_row = original_regions[i][1]
        prev_cols = set(original_regions[i - 1][2:])
        current_cols = set(original_regions[i][2:])

        # 找出相邻两行列索引的最小和最大值
        min_col = min(min(prev_cols), min(current_cols))
        max_col = max(max(prev_cols), max(current_cols))

        # 填充上一行和当前行之间的空缺行
        for row in range(prev_row + 1, current_row):
            connected_regions.append((0, row, *range(min_col, max_col + 1)))

        # 添加当前行
        connected_regions.append(original_regions[i])

# 标记非活动单元格
for region in connected_regions:
    lay = region[0]
    row = region[1]
    for col in region[2:]:
        idomain[lay, row, col] = -1

# 让 20 到 25 行保持活动状态
for row in range(14, 23):
    idomain[0, row, 48:] = 1
    
laytyp = 0  # 含水层类型

# 初始化 hk 数组（原尺寸）
hk_original = np.ones((nlay, nrow, ncol), dtype=np.float32)

# 原位置信息赋值
hk_original[:, 0, 3:8] = 0.0004 * 864
hk_original[:, 1, 2:10] = 0.0004 * 864
hk_original[:, 2, 2:9] = 0.0004 * 864
hk_original[:, 3, 1:8] = 0.0004 * 864
hk_original[:, 4, 0:7] = 0.0004 * 864
hk_original[:, 5, 1:6] = 0.0004 * 864
hk_original[:, 6, 1:5] = 0.0004 * 864
hk_original[:, 7, 2:4] = 0.0004 * 864

hk_original[:, 2, 9:15] = 0.0002 * 864
hk_original[:, 3, 8:16] = 0.0002 * 864
hk_original[:, 4, 7:21] = 0.0002 * 864
hk_original[:, 5, 7:23] = 0.0002 * 864
hk_original[:, 6, 10:23] = 0.0002 * 864
hk_original[:, 7, 14:21] = 0.0002 * 864
hk_original[:, 8, 16:19] = 0.0002 * 864

hk_original[:, 5, 6] = 0.0001 * 864
hk_original[:, 6, 5:10] = 0.0001 * 864
hk_original[:, 7, 4:12] = 0.0001 * 864
hk_original[:, 8, 3:11] = 0.0001 * 864
hk_original[:, 9, 4:10] = 0.0001 * 864

hk_original[:, 7, 12:14] = 0.0003 * 864
hk_original[:, 8, 11:16] = 0.0003 * 864
hk_original[:, 9, 10:17] = 0.0003 * 864
hk_original[:, 10, 12:18] = 0.0003 * 864
hk_original[:, 11, 13:19] = 0.0003 * 864
hk_original[:, 12, 16:19] = 0.0003 * 864
hk_original[:, 13, 17:20] = 0.0003 * 864
hk_original[:, 14, 19] = 0.0003 * 864

hk_original[:, 6, 23] = 0.0007 * 864
hk_original[:, 7, 21:25] = 0.0007 * 864
hk_original[:, 8, 19:25] = 0.0007 * 864
hk_original[:, 9, 16:25] = 0.0007 * 864
hk_original[:, 10, 17:25] = 0.0007 * 864
hk_original[:, 11, 18:24] = 0.0007 * 864
hk_original[:, 12, 18:23] = 0.0007 * 864
hk_original[:, 13, 19:22] = 0.0007 * 864
hk_original[:, 14, 20] = 0.0007 * 864

# 初始化新的 hk 数组（新尺寸）
hk = np.ones((nlay, nrow, ncol), dtype=np.float32)

# 计算行列缩放比例
row_scale = 2
col_scale = 2

# 遍历原 hk 数组，按比例复制到新数组
for lay in range(nlay):
    for row in range(15):
        for col in range(25):
            new_row = row * row_scale
            new_col = col * col_scale
            hk[lay, new_row:new_row + row_scale, new_col:new_col + col_scale] = hk_original[lay, row, col]

num_stress_periods = 30  # 应力期数量
# 河流边界（关键修改部分）
# R变化的情况
rbot = np.linspace(90., 80.25, num=nrow)
#rstage = np.linspace(90.1, 81.25, num=nrow)
riv_spd = {}
for sp in range(num_stress_periods):
    rstage = np.linspace(90.1, 81.25, num=nrow)
    riv_spd[sp] = []
    for col in [14]:
        for row in range(2, 11):      
            s = rstage[row - 2]
            b = rbot[row - 2]  # 河底高程根据行号取值
            riv_spd[sp].append([0, row, col, s, 50, b])


# 应力周期长度 6 个月
perlen = 6 * 30  #定义单个应力期（Stress Period）的长度
nstp = 1  #表示每个应力期仅使用一个时间步，即全应力期长度（180天）作为一个时间步
tsmult = 1  #=1：时间步长恒定（每步长度相同）。>1：时间步长逐步增大（如指数增长）。<1：时间步长逐步减小（较少使用）。


#定义观测井位置
obs_wells = [
    [0, 6, 8],  # 层，行，列
    [0, 4, 16],
    [0, 4, 28], 
    [0, 14, 30],
    [0, 12, 18],  # 层，行，列
    [0, 18, 12],
    [0, 14, 24], 
    [0, 26, 38],
    [0, 18, 38], 
    [0, 20, 46]
    ]

# 渗透系数列表
permeability_values = [0.0004 * 864, 0.0002 * 864, 0.0001 * 864, 0.0003 * 864, 0.0007 * 864]

# 初始化垂向补给率数组
recharge = np.zeros((nlay, nrow, ncol))

# 初始化蒸发率数组
evaporation = np.zeros((nlay, nrow, ncol))

# 创建 rch 和 evt 模块的应力期数据
rch_spd = {}
evt_spd = {}
for sp in range(nper):
    if sp % 2 == 1:  # 丰水期
        # 生成丰水期垂向补给率随机变量（范围除以100）
        recharge_factors = {
            0.0004 * 864: np.random.uniform(0.016 * 1.1 / 100, 0.016 * 1.3 / 100),
            0.0002 * 864: np.random.uniform(0.012 * 1.1 / 100, 0.012 * 1.3 / 100),
            0.0001 * 864: np.random.uniform(0.008 * 1.1 / 100, 0.008 * 1.3 / 100),
            0.0003 * 864: np.random.uniform(0.014 * 1.1 / 100, 0.014 * 1.3 / 100),
            0.0007 * 864: np.random.uniform(0.018 * 1.1 / 100, 0.018 * 1.3 / 100)
        }
        # 生成丰水期蒸发率随机变量（范围除以100）
        evaporation_factors = {
            0.0004 * 864: np.random.uniform(0.002 * 0.7 / 100, 0.002 * 0.9 / 100),
            0.0002 * 864: np.random.uniform(0.003 * 0.7 / 100, 0.003 * 0.9 / 100),
            0.0001 * 864: np.random.uniform(0.001 * 0.7 / 100, 0.001 * 0.9 / 100),
            0.0003 * 864: np.random.uniform(0.004 * 0.7 / 100, 0.004 * 0.9 / 100),
            0.0007 * 864: np.random.uniform(0.005 * 0.7 / 100, 0.005 * 0.9 / 100)
        }
    else:  # 枯水期
        # 生成枯水期垂向补给率随机变量（范围除以100）
        recharge_factors = {
            0.0004 * 864: np.random.uniform(0.016 * 0.7 / 100, 0.016 * 0.9 / 100),
            0.0002 * 864: np.random.uniform(0.012 * 0.7 / 100, 0.012 * 0.9 / 100),
            0.0001 * 864: np.random.uniform(0.008 * 0.7 / 100, 0.008 * 0.9 / 100),
            0.0003 * 864: np.random.uniform(0.014 * 0.7 / 100, 0.014 * 0.9 / 100),
            0.0007 * 864: np.random.uniform(0.018 * 0.7 / 100, 0.018 * 0.9 / 100)
        }
        # 生成枯水期蒸发率随机变量（范围除以100）
        evaporation_factors = {
            0.0004 * 864: np.random.uniform(0.002 * 1.1 / 100, 0.002 * 1.3 / 100),
            0.0002 * 864: np.random.uniform(0.003 * 1.1 / 100, 0.003 * 1.3 / 100),
            0.0001 * 864: np.random.uniform(0.001 * 1.1 / 100, 0.001 * 1.3 / 100),
            0.0003 * 864: np.random.uniform(0.004 * 1.1 / 100, 0.004 * 1.3 / 100),
            0.0007 * 864: np.random.uniform(0.005 * 1.1 / 100, 0.005 * 1.3 / 100)
        }

    current_rch = recharge.copy()
    current_evt = evaporation.copy()

    for k in permeability_values:
        rch_mask = np.isclose(hk, k)
        current_rch[rch_mask] = recharge_factors[k]

        evt_mask = np.isclose(hk, k)
        current_evt[evt_mask] = evaporation_factors[k]

    rch_spd[sp] = current_rch
    evt_spd[sp] = []
    for lay in range(nlay):
        for row in range(nrow):
            for col in range(ncol):
                if current_evt[lay, row, col] > 0:
                    lay = int(lay)
                    row = int(row)
                    col = int(col)
                    rate = float(current_evt[lay, row, col])
                    # 假设 surface 为 100.0（可根据实际情况修改）
                    surface = 100.0
                    # 假设 depth 为 0.0（可根据实际情况修改）
                    depth = 5.0
                    evt_spd[sp].append([lay, row, col, surface, rate, depth])


# Solver parameters
nouter, ninner = 100, 300
hclose, rclose, relax = 1e-6, 1e-6, 1.0

ss = np.full((nlay, nrow, ncol), 1e-5)  # 储水系数
if __name__ == "__main__":
    n_Q = 4  # Q 的样本数量
    d_Q = 10  # Q 的维度
    bounds_Q = [(50, 100)] * d_Q  # Q 的边界

    n_R_initial = 1  # 初始水位采样数量
    d_Ri = 20  # R 的维度
    bounds_Ri = [(90.10, 92.50)] * d_Ri  # R 的边界

    n_R_final = 1  # 末尾水位采样数量
    d_Rf = 20  # R 的维度
    bounds_Rf = [(81.25, 82.50)] * d_Rf  # R 的边界

    # 重复运行200次
    for run in range(100):
        # 生成初始拉丁超立方设计
        X_init_Q = generate_initial_lhs(n_Q, d_Q, bounds_Q)
        X_init_R_initial = generate_initial_lhs(n_R_initial, d_Ri, bounds_Ri)
        X_init_R_final = generate_initial_lhs(n_R_final, d_Rf, bounds_Rf)

        # 初始化退火参数
        T0, alpha, Nmax = initialize_annealing_parameters()

        # 单独对 Q 进行采样优化
        X_Q_optimized = simulated_annealing(X_init_Q, T0, alpha, Nmax, mode='Q_only')
        Q_optimized = X_Q_optimized

        # 单独对初始水位 R 进行采样优化
        X_R_initial_optimized = simulated_annealing(X_init_R_initial, T0, alpha, Nmax, mode='R_only')
        initial_water_level = X_R_initial_optimized

        # 单独对末尾水位 R 进行采样优化
        X_R_final_optimized = simulated_annealing(X_init_R_final, T0, alpha, Nmax, mode='R_only')
        final_water_level = X_R_final_optimized

        # 确保初始水位数组中每个元素都大于末尾水位数组对应位置的元素
        for i in range(initial_water_level.shape[1]):
            if initial_water_level[0, i] < final_water_level[0, i]:
                initial_water_level[0, i], final_water_level[0, i] = final_water_level[0, i], initial_water_level[0, i]

        output_path = os.path.join(output_dir, f"optimized_Q_values_{run}.txt")
        np.savetxt(output_path, Q_optimized)
        output_path = os.path.join(output_dir, f"optimized_R_initial_values_{run}.txt")
        np.savetxt(output_path, initial_water_level)
        output_path = os.path.join(output_dir, f"optimized_R_final_values_{run}.txt")
        np.savetxt(output_path, final_water_level)

        # 井采样部分
        n_wells = 10
        # 井采样的维度为 3（层、行、列）
        d_well = 3
        bounds_well = [(0, nlay - 1), (0, nrow - 1), (0, ncol - 1)]
        # 生成初始拉丁超立方设计
        X_init_well = generate_initial_lhs(n_wells, d_well, bounds_well)
        # 对井的位置进行取整
        X_init_well = np.round(X_init_well).astype(int)
        # 过滤掉不在活动单元格内的位置
        valid_well_locations = []
        for location in X_init_well:
            lay, row, col = location
            if idomain[lay, row, col] == 1:
                valid_well_locations.append((lay, row, col))
        while len(valid_well_locations) < n_wells:
            lay = np.random.randint(0, nlay)
            row = np.random.randint(0, nrow)
            col = np.random.randint(0, ncol)
            if idomain[lay, row, col] == 1:
                location = (lay, row, col)
                if location not in valid_well_locations:
                    valid_well_locations.append(location)
        # 将有效位置转换为 numpy 数组
        X_init_well = np.array(valid_well_locations)

        # 初始化退火参数
        T0, alpha, Nmax = initialize_annealing_parameters()

        # 对井的位置进行退火拉丁超立方采样优化
        X_well_optimized = simulated_annealing(X_init_well, T0, alpha, Nmax, mode='both')
        # 再次对优化后的位置进行取整
        X_well_optimized = np.round(X_well_optimized).astype(int)
        well_locations = []
        for location in X_well_optimized:
            lay, row, col = location
            if idomain[lay, row, col] == 1:
                well_locations.append((lay, row, col))
        while len(well_locations) < n_wells:
            lay = np.random.randint(0, nlay)
            row = np.random.randint(0, nrow)
            col = np.random.randint(0, ncol)
            if idomain[lay, row, col] == 1:
                location = (lay, row, col)
                if location not in well_locations:
                    well_locations.append(location)

        c_values = np.array([0.5371, 0.9508, 0.8966, 0.9340, 
                0.6388 , 0.5806, 0.6135, 0.6934, 0.9771, 0.8470 ])
        c_values = c_values*20
        wel_spd = {
            i: [
                [lay, row, col, 0, 0] for lay, row, col in well_locations
            ] for i in range(4)
        }

        # 将Q采样数据代入前四个应力周期的wel_spd
        for stress_period in range(min(4, len(Q_optimized))):
            q_values = Q_optimized[stress_period]
            for i, well in enumerate(wel_spd[stress_period]):
                well[3] = q_values[i]
                well[4] = c_values[i]

        well_locations_array = np.array(well_locations)
        output_path_wells = os.path.join(output_dir, f"well_locations_{run}.txt")
        np.savetxt(output_path_wells, well_locations_array)
        
        # 2. 创建模拟
        sim_ws = os.path.join('confined_conc_models', f'{sim_name}_{run}')  # 定义模型的工作目录（工作空间）
        sim = flopy.mf6.MFSimulation(sim_name=f'{sim_name}_{run}', sim_ws=sim_ws, exe_name='../modflow/mf6.6.1_linux/bin/mf6')  # 创建MODFLOW 6模拟的主对象

        # 应力周期长度 6 个月
        tdis_ds = [(perlen, nstp, tsmult) for _ in range(nper)]  # 生成时间离散化数据，包含nper=20个元组的列表。
        flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_ds, time_units=time_units)  # （对象，总应力期数量，列表每个元素为(perlen, nstp, tsmult)元组，时间单位）

        # 3. 建立模型
        gwfname =  f"gwf_{sim_name}_{run}"
        gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, newtonoptions="NEWTON UNDER_RELAXATION", 
                                   save_flows=True)  # newtonoptions="NEWTON UNDER_RELAXATION"​​牛顿-拉夫森法的松弛选项，用于改善非线性问题的收敛性
        # 渗流模型求解参数
        imsgwf = flopy.mf6.ModflowIms(
            sim,
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename="{}.ims".format(gwfname),
          )
        sim.register_ims_package(imsgwf, [gwf.name])

        flopy.mf6.ModflowGwfdis(gwf, length_units=length_units, nlay=nlay, nrow=nrow,
                                ncol=ncol, delr=delr, delc=delc, top=top, botm=botm, idomain=idomain)  # 配置模型的网格离散化参数

        flopy.mf6.ModflowGwfnpf(gwf, icelltype=icelltype, k=hk)  # 定义含水层的水力传导系数（渗透系数）和单元类型
        flopy.mf6.ModflowGwfsto(gwf, ss=ss, transient={0: True})
        flopy.mf6.ModflowGwfic(gwf, strt=strt)  # 设置模型的初始水头分布。

        flopy.mf6.ModflowGwfriv(gwf, stress_period_data=riv_spd, pname="RIV-1", print_flows=True, save_flows=True)  # 定义河流边界条件。

        flopy.mf6.ModflowGwfwel(gwf, stress_period_data=wel_spd, auxiliary=['CONCENTRATION'], 
                                pname="WEL-1", save_flows=True)  # 定义抽水/注水井

        flopy.mf6.ModflowGwfrcha(gwf, recharge=rch_spd)  # 定义垂向补给（如降水入渗）
        flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd)  # 定义定水头边界（如河流或湖泊）
        flopy.mf6.ModflowGwfevt(gwf, stress_period_data=evt_spd, pname="EVT-1", print_flows=True, save_flows=True)
       # 输出数据
       # headfile = os.path.join(run_output_dir, f"gwf_{sim_name}_{run+1}.hds")
        #budgetfile = os.path.join(run_output_dir, f"gwf_{sim_name}_{run+1}.cbb")
        
       # concentrationfile = os.path.join(run_output_dir, f"gwt_{sim_name}_{run+1}.ucn")
        #gwt_budgetfile = os.path.join(run_output_dir, f"gwt_{sim_name}_{run+1}.cbc")
        
        headfile = f"{sim_name}_{run}.hds"
        head_filerecord = [headfile]
        budgetfile = f"{sim_name}_{run}.bud"
        budget_filerecord = [budgetfile]
        saverecord = [("HEAD", "ALL"), ("BUDGET", "ALL")]
        flopy.mf6.ModflowGwfoc(gwf, saverecord=saverecord, head_filerecord=head_filerecord,
                               budget_filerecord=budget_filerecord, printrecord=saverecord)

    # 创建观测井文件
        obs_data = {
            f'obs_well_{i+1}.csv': [
                (f'well_{i+1}', 'HEAD', (lay, row, col))
            ]
            for i, (lay, row, col) in enumerate(obs_wells)
        }

        flopy.mf6.ModflowUtlobs(
              gwf,
              digits=10,
              print_input=True,
              continuous=obs_data,          
        )

    # 创建运移模型并与流动模型耦合
        gwtname = f"gwt_{sim_name}_{run}"
        gwt = flopy.mf6.ModflowGwt(sim, modelname=gwtname, model_nam_file=f"{gwtname}.nam")

        # 溶质运移模型求解参数
        imsgwt = flopy.mf6.ModflowIms(
            sim,
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename="{}.ims".format(gwtname),
        )
        sim.register_ims_package(imsgwt, [gwt.name])

    # 运移模型的时间离散化（需要与GWF模型同步）
        flopy.mf6.ModflowGwtdis(gwt, nlay=nlay, nrow=nrow, ncol=ncol,
                                delr=delr, delc=delc, top=top, botm=botm, idomain=idomain)

    # 运移模型的初始条件
        initial_concentration = 0.0  # 初始浓度设为0
        flopy.mf6.ModflowGwtic(gwt, strt=initial_concentration)

    # 弥散系数设置
        dsp = flopy.mf6.ModflowGwtdsp(gwt,
                            alh=40.0,  # 纵向弥散度
                            ath1=4.0,  # 横向弥散度
                            atv=0.1,   # 垂向弥散度
                            diffc=0.0) # 分子扩散系数
    # 运移过程设置
        mst = flopy.mf6.ModflowGwtmst(gwt, porosity=0.3)  # 孔隙度设为0.3
        sourcerecarray = [('WEL-1', 'AUX', 'CONCENTRATION')]
        
        flopy.mf6.ModflowGwtssm(
              gwt,
              sources = sourcerecarray,
              print_flows=True,
              filename=f"{gwtname}.ssm"
         )


    # 观测井配置（在原有观测井之后添加）
        gwt_obs_package_name = f"gwt_{sim_name}_{run+1}_obs"
        gwt_obs = [
            (f"conc_well_{i}", "CONCENTRATION", (lay, row, col))
            for i, (lay, row, col) in enumerate(obs_wells)
        ]

        flopy.mf6.ModflowUtlobs(
              gwt,
              pname=gwt_obs_package_name,
              digits=10,
              print_input=True,
              continuous=gwt_obs,

          )


   # 创建每次运行对应的独立输出文件夹
        run_output_dir = os.path.join(output_dir, f"run_{run + 1}")
        os.makedirs(run_output_dir, exist_ok=True)

    # 输出控制
        concentrationfile = f"{sim_name}_{run}.ucn"  # 绝对路径
        gwt_budgetfile = f"{sim_name}_{run}.cbc"
        flopy.mf6.ModflowGwtoc(gwt,
                               concentration_filerecord = [concentrationfile],
                               budget_filerecord = [gwt_budgetfile],
                               saverecord=[("CONCENTRATION", "ALL"), 
                                           ("BUDGET", "ALL")])
    # 耦合模型
        flopy.mf6.ModflowGwfgwt(sim, exgtype='GWF6-GWT6', exgmnamea=gwfname, exgmnameb=gwtname,filename="{}.gwfgwt".format(sim_name))
        # 运行模型
        sim.write_simulation()
        success, buff = sim.run_simulation()
        if not success:
            raise Exception(f"模型运行失败 for run {run}")

已跳过重复的定水头单元格: (0, 2, 4)
已跳过重复的定水头单元格: (0, 4, 4)
已跳过重复的定水头单元格: (0, 6, 2)
已跳过重复的定水头单元格: (0, 28, 43)
已跳过重复的定水头单元格: (0, 26, 45)
已跳过重复的定水头单元格: (0, 24, 47)
writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing solution package ims_0...
  writing package con.gwfgwt...
  writing model gwf_con_0...
    writing model name file...
    writing package dis...
    writing package npf...
    writing package sto...
    writing package ic...
    writing package riv-1...
INFORMATION: maxbound in ('gwf6', 'riv', 'dimensions') changed to 9 based on size of stress_period_data
    writing package wel-1...
INFORMATION: maxbound in ('gwf6', 'wel', 'dimensions') changed to 10 based on size of stress_period_data
    writing package rcha_0...
    writing package chd_0...
INFORMATION: maxbound in ('gwf6', 'chd', 'dimensions') changed to 31 based on size of stress_period_data
    writing package evt-1...
INFORMATION: maxbound in 

In [5]:
import os
import numpy as np
import pandas as pd
import traceback
from datetime import datetime
import flopy

def extract_all_grid_data(gwf, gwt, workspace, model_name):
    """提取单个模型所有网格数据，包括CHD标记和GWT浓度（所有层）"""
    
    def get_flattened(data, t, lay, default_shape):
        """安全处理多维数组到1D的转换，支持多层数据"""
        if data is None:
            return np.zeros(default_shape)
        
        try:
            if data.ndim == 4:  # (time, lay, row, col)
                return data[t, lay].flatten() if t < data.shape[0] else np.zeros_like(data[0, 0]).flatten()
            elif data.ndim == 3:  # (time, row, col)
                return data[t].flatten() if t < data.shape[0] else np.zeros_like(data[0]).flatten()
            elif data.ndim == 2:  # (row, col)
                return data.flatten()
            else:  # 标量
                return np.full(default_shape, float(data))
        except Exception as e:
            print(f"警告: 数据展平失败 (t={t}, lay={lay}), 使用零值替代. 错误: {str(e)}")
            return np.zeros(default_shape)

    try:
        print(f"\n{'='*50}\n开始提取模型 {model_name} @ {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n{'='*50}")
        
        # 1. 获取基础网格信息
        print("步骤1/9: 获取模型对象...")
        dis = gwf.dis
        npf = gwf.npf
        
        nlay = int(dis.nlay.array)
        nrow = int(dis.nrow.array)
        ncol = int(dis.ncol.array)
        grid_size = nrow * ncol
        print(f"网格维度: nlay={nlay}, nrow={nrow}, ncol={ncol}")

        # 2. 处理网格参数
        print("步骤2/9: 处理网格参数...")
        delr = dis.delr.array.mean() if dis.delr.array.size > 1 else float(dis.delr.array)
        delc = dis.delc.array.mean() if dis.delc.array.size > 1 else float(dis.delc.array)
        top = dis.top.array.flatten() if isinstance(dis.top.array, np.ndarray) else np.full(grid_size, float(dis.top.array))
        botm = dis.botm.array  # 获取所有层的底部高程

        # 3. 创建基础DataFrame
        print("步骤3/9: 创建基础DataFrame...")
        x_coords = np.linspace(delr/2, ncol*delr - delr/2, ncol)
        y_coords = np.linspace(delc/2, nrow*delc - delc/2, nrow)
        xx, yy = np.meshgrid(x_coords, y_coords)
        
        df_base = pd.DataFrame({
            "row": np.repeat(np.arange(nrow), ncol),
            "col": np.tile(np.arange(ncol), nrow),
            "x": xx.flatten(),
            "y": yy.flatten(),
            "top": top,
            "K": get_flattened(npf.k.array, 0, 0, grid_size),  # 初始使用第0层
            "model_name": model_name,
            "nlay": nlay
        })

        # 4. 提取水头数据
        print("步骤4/9: 提取水位数据...")
        head_obj = gwf.output.head()
        if head_obj is None:
            raise ValueError("未找到.head()输出，请确认OC包设置正确")
        all_heads = head_obj.get_alldata()
        nper = all_heads.shape[0]
        print(f"水位数据: {nper}时间步, 形状{all_heads.shape}")

        # 5. 提取污染物浓度数据
        print("步骤5/9: 提取污染物浓度数据...")
        conc_data = None
        if gwt is not None:
            try:
                conc_obj = gwt.output.concentration()
                if conc_obj is not None:
                    conc_data = conc_obj.get_alldata()
                    print(f"浓度数据: {conc_data.shape[0]}时间步, 形状{conc_data.shape}")
                else:
                    print("警告: 未找到浓度输出对象")
            except Exception as e:
                print(f"浓度数据提取警告: {str(e)}")
        else:
            print("警告: 未找到GWT模型")

        # 6. 提取其他包数据（全部修改为支持多层）
        print("步骤6/9: 提取其他包数据...")
        package_data = {}
        
        # 处理CHD数据（多层）
        if hasattr(gwf, 'chd'):
            print("  - 提取CHD数据...")
            chd_spd = gwf.chd.stress_period_data.get_data()
            
            chd_mask = np.zeros((nper, nlay, nrow, ncol))
            
            for t in range(nper):
                if t in chd_spd:
                    for rec in chd_spd[t]:
                        cellid = rec['cellid']
                        if isinstance(cellid, tuple) and len(cellid) == 3:
                            lay, row, col = cellid[0], cellid[1], cellid[2]
                        elif isinstance(cellid, tuple) and len(cellid) == 2:
                            row, col = cellid[0], cellid[1]
                            lay = 0  # 如果没有指定层，默认第0层
                        else:
                            node = cellid
                            lay, row, col = np.unravel_index(node-1, (nlay, nrow, ncol))
                        
                        chd_mask[t, lay, row, col] = 1
            
            package_data['chd_mask'] = chd_mask
            print(f"    CHD数据点: {sum(len(v) for v in chd_spd.values())}")

        # 处理补给(RCH)数据（多层）
        if hasattr(gwf, 'rcha'):
            print("  - 提取补给(RCH)数据...")
            rch = gwf.rcha.recharge.array
            # 将RCH数据扩展到所有层
            if rch.ndim == 2:  # (row, col)
                rch = np.broadcast_to(rch, (nper, nlay, nrow, ncol))
            elif rch.ndim == 3:  # (time, row, col)
                rch = np.broadcast_to(rch[:, np.newaxis, :, :], (nper, nlay, nrow, ncol))
            package_data['recharge'] = rch
            print(f"    RCH形状: {rch.shape}")

        # 处理蒸发(EVT)数据（多层）
        if hasattr(gwf, 'evt'):
            print("  - 提取蒸发(EVT)数据...")
            try:
                evt = gwf.evt
                evt_spd = evt.stress_period_data.get_data()
                evt_data = np.zeros((nper, nlay, nrow, ncol))
                
                for t in range(nper):
                    if t in evt_spd:
                        for rec in evt_spd[t]:
                            cellid = rec['cellid']
                            if isinstance(cellid, tuple) and len(cellid) == 3:
                                lay, row, col = cellid[0], cellid[1], cellid[2]
                            elif isinstance(cellid, tuple) and len(cellid) == 2:
                                row, col = cellid[0], cellid[1]
                                lay = 0  # 默认第0层
                            rate = rec['rate']
                            surface = rec['surface'] if 'surface' in rec.dtype.names else 100.0
                            depth = rec['depth'] if 'depth' in rec.dtype.names else 5.0
                            evt_data[t, lay, row, col] = rate
                
                package_data['ET'] = evt_data
                print(f"    EVT数据形状: {evt_data.shape}")
                
            except Exception as e:
                print(f"    EVT数据提取错误: {str(e)}")
                traceback.print_exc()

        # 处理河流(RIV)数据（多层）
        if hasattr(gwf, 'riv'):
            print("  - 提取河流(RIV)数据...")
            riv_spd = gwf.riv.stress_period_data.get_data()
            
            riv_stage = np.zeros((nper, nlay, nrow, ncol))
            riv_cond = np.zeros((nper, nlay, nrow, ncol))
            riv_rbot = np.zeros((nper, nlay, nrow, ncol))
            
            for t in range(nper):
                if t in riv_spd:
                    for rec in riv_spd[t]:
                        cellid = rec['cellid']
                        if isinstance(cellid, tuple) and len(cellid) == 3:
                            lay, row, col = cellid[0], cellid[1], cellid[2]
                        elif isinstance(cellid, tuple) and len(cellid) == 2:
                            row, col = cellid[0], cellid[1]
                            lay = 0  # 默认第0层
                        riv_stage[t, lay, row, col] = rec['stage']
                        riv_cond[t, lay, row, col] = rec['cond']
                        riv_rbot[t, lay, row, col] = rec['rbot']
            
            package_data.update({
                'river_stage': riv_stage,
                'river_cond': riv_cond,
                'river_rbot': riv_rbot
            })
            print(f"    RIV数据点: {sum(len(v) for v in riv_spd.values())}")
                
        # 处理抽水井(WEL)数据（多层）
        if hasattr(gwf, 'wel'):
            print("  - 提取抽水井(WEL)数据...")
            wel_spd = gwf.wel.stress_period_data.get_data()
            
            wel_rate = np.zeros((nper, nlay, nrow, ncol))
            wel_cells = set()
            
            for t in range(nper):
                if t in wel_spd:
                    for rec in wel_spd[t]:
                        cellid = rec['cellid']
                        if isinstance(cellid, tuple) and len(cellid) == 3:
                            lay, row, col = cellid[0], cellid[1], cellid[2]
                        else:
                            node = cellid
                            lay, row, col = np.unravel_index(node-1, (nlay, nrow, ncol))
                        
                        rate = rec['q']
                        wel_rate[t, lay, row, col] = rate
                        wel_cells.add((lay, row, col))
            
            package_data['well_rate'] = wel_rate
            print(f"    WEL数据点: {len(wel_cells)}个井位置")
            
            wel_mask = np.zeros((nper, nlay, nrow, ncol))
            for lay, row, col in wel_cells:
                wel_mask[:, lay, row, col] = 1
            package_data['well_mask'] = wel_mask

        # 7. 合并所有时间步和层的数据
        print("步骤7/9: 合并时间步和层数据...")
        full_dfs = []
        min_time_steps = min([nper] + [d.shape[0] for d in package_data.values() if hasattr(d, 'shape')])
        
        for t in range(min_time_steps):
            for lay in range(nlay):
                df_t = df_base.copy()
                df_t["time_step"] = t
                df_t["layer"] = lay  # 添加层号
                df_t["bottom"] = botm[lay].flatten() if botm.ndim > 1 else np.full(grid_size, float(botm[lay]))
                df_t["head"] = get_flattened(all_heads, t, lay, grid_size)
                
                # 添加浓度数据
                if conc_data is not None and t < conc_data.shape[0]:
                    df_t["concentration"] = get_flattened(conc_data, t, lay, grid_size)
                else:
                    df_t["concentration"] = np.zeros(grid_size)
                
                # 添加其他包数据
                for name, data in package_data.items():
                    df_t[name] = get_flattened(data, t, lay, grid_size)
                
                full_dfs.append(df_t)

        # 8. 返回数据不保存
        print("步骤8/9: 准备返回数据...")
        full_data = pd.concat(full_dfs, ignore_index=True)
        HEAD_THRESHOLD = 1e10  
        cleaned_data = full_data[full_data['head'] < HEAD_THRESHOLD]
        cleaned_data.reset_index(drop=True, inplace=True)

        # 9. 验证和返回
        print("步骤9/9: 验证数据完整性...")
        required_cols = ['row', 'col', 'layer', 'x', 'y', 'head', 'time_step', 'concentration', 'nlay']
        missing = [col for col in required_cols if col not in full_data.columns]
        if missing:
            raise ValueError(f"缺失关键列: {missing}")
            
        print(f"\n{'='*50}\n提取成功! 总数据量: {len(full_data)}行")
        print(f"包含的包数据: {list(package_data.keys())}")
        print(f"时间步范围: 0-{full_data['time_step'].max()}, 层范围: 0-{nlay-1}")
        print(f"内存占用: {full_data.memory_usage().sum()/1e6:.1f}MB\n{'='*50}")
        cleaned_data['concentration'] = cleaned_data['concentration'].clip(lower=0)
        return cleaned_data

    except Exception as e:
        print(f"\n{'!'*50}\n错误详情:\n{traceback.format_exc()}\n{'!'*50}")
        raise
def process_models_directory(root_dir, output_csv):
    """
    遍历目录处理所有MODFLOW6模型

    参数:
        root_dir: 包含多个模型目录的根目录
        output_csv: 最终合并的输出CSV文件路径
    """
    all_data = []

    # 遍历根目录
    for dirpath, dirnames, filenames in os.walk(root_dir):
        # 检查是否是MODFLOW6模型目录 (包含.nam文件)
        nam_files = [f for f in filenames if f.endswith('.nam')]
        
        if nam_files:
            model_name = os.path.basename(dirpath)
            print(f"\n{'#'*80}\n发现MODFLOW6模型: {model_name}\n目录: {dirpath}\n{'#'*80}")
            
            try:
                # 加载GWF模型
                sim = flopy.mf6.MFSimulation.load(sim_ws=dirpath)
                gwf = sim.get_model()  # 获取第一个模型
                
                # 尝试加载GWT模型
                gwt = None
                try:
                    gwt_models = [m for m in sim.model_names if 'gwt' in m.lower()]
                    if gwt_models:
                        gwt = sim.get_model(gwt_models[0])
                        print(f"找到GWT模型: {gwt_models[0]}")
                    else:
                        print(f"未找到GWT模型，模型名称列表: {sim.model_names}")
                except Exception as e:
                    print(f"GWT模型加载失败: {str(e)}")
                    traceback.print_exc()
                
                # 提取数据
                model_data = extract_all_grid_data(gwf, gwt, dirpath, model_name)
                if not model_data.empty:
                    all_data.append(model_data)
                
                # 释放内存
                del sim, gwf, gwt, model_data
                
            except Exception as e:
                print(f"处理模型 {model_name} 失败: {str(e)}")
                traceback.print_exc()
                continue
    
    # 合并所有模型数据
    if all_data:
        print(f"\n{'='*80}\n合并所有模型数据...")
        combined_data = pd.concat(all_data, ignore_index=True)
        
        # 保存合并后的数据
        print(f"总数据量: {len(combined_data)}行")
        print(f"包含的模型: {combined_data['model_name'].unique()}")
        
        # 分块保存大数据
        if len(combined_data) > 100000000:  # 1百万行分块
            chunks = [combined_data.iloc[i:i+100000000] for i in range(0, len(combined_data), 100000000)]
            for i, chunk in enumerate(chunks):
                chunk_path = output_csv.replace(".csv", f"_part{i+1}.csv")
                chunk.to_csv(chunk_path, index=False)
                print(f"保存分块 {i+1} 到 {chunk_path} (行数: {len(chunk)})")
        else:
            combined_data.to_csv(output_csv, index=False)
            print(f"保存合并数据到 {output_csv}")
        
        return combined_data
    else:
        print("未找到任何有效的MODFLOW6模型数据")
        return None

# 使用示例
if __name__ == "__main__":
    # 设置根目录和输出文件路径
    models_root_dir = "confined_conc_models"  # 替换为你的模型根目录
    output_file = "conc_confine.csv"  # 输出文件路径
    
    # 处理所有模型
    process_models_directory(models_root_dir, output_file)


################################################################################
发现MODFLOW6模型: con_9
目录: confined_conc_models/con_9
################################################################################
loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package npf...
    loading package sto...
    loading package ic...
    loading package riv...
    loading package wel...
    loading package rch...
    loading package chd...
    loading package evt...
    loading package oc...
    loading package obs...
  loading model gwt6...
    loading package dis...
    loading package ic...
    loading package dsp...
    loading package mst...
    loading package ssm...
    loading package obs...
    loading package oc...
  loading exchange package gwf-gwt_exg_0...
  loading solution package gwf_con_9...
  loading solution package gwt_con_9...
找到GWT模型: gwt_con_9

开始提取模型 con_9 @ 2025-08-01 17:2

/tmp/ipykernel_480897/2557538960.py:264: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['concentration'] = cleaned_data['concentration'].clip(lower=0)


    loading package wel...
    loading package rch...
    loading package chd...
    loading package evt...
    loading package oc...
    loading package obs...
  loading model gwt6...
    loading package dis...
    loading package ic...
    loading package dsp...
    loading package mst...
    loading package ssm...
    loading package obs...
    loading package oc...
  loading exchange package gwf-gwt_exg_0...
  loading solution package gwf_con_34...
  loading solution package gwt_con_34...
找到GWT模型: gwt_con_34

开始提取模型 con_34 @ 2025-08-01 17:22:33
步骤1/9: 获取模型对象...
网格维度: nlay=1, nrow=30, ncol=50
步骤2/9: 处理网格参数...
步骤3/9: 创建基础DataFrame...
步骤4/9: 提取水位数据...
水位数据: 30时间步, 形状(30, 1, 30, 50)
步骤5/9: 提取污染物浓度数据...
浓度数据: 30时间步, 形状(30, 1, 30, 50)
步骤6/9: 提取其他包数据...
  - 提取CHD数据...
    CHD数据点: 31
  - 提取补给(RCH)数据...
    RCH形状: (30, 1, 30, 50)
  - 提取蒸发(EVT)数据...
    EVT数据形状: (30, 1, 30, 50)
  - 提取河流(RIV)数据...
    RIV数据点: 270
  - 提取抽水井(WEL)数据...
    WEL数据点: 10个井位置
步骤7/9: 合并时间步和层数据...
步骤8/9: 准备返回数据...
步骤9/

/tmp/ipykernel_480897/2557538960.py:264: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['concentration'] = cleaned_data['concentration'].clip(lower=0)


    loading package wel...
    loading package rch...
    loading package chd...
    loading package evt...
    loading package oc...
    loading package obs...
  loading model gwt6...
    loading package dis...
    loading package ic...
    loading package dsp...
    loading package mst...
    loading package ssm...
    loading package obs...
    loading package oc...
  loading exchange package gwf-gwt_exg_0...
  loading solution package gwf_con_28...
  loading solution package gwt_con_28...
找到GWT模型: gwt_con_28

开始提取模型 con_28 @ 2025-08-01 17:22:39
步骤1/9: 获取模型对象...
网格维度: nlay=1, nrow=30, ncol=50
步骤2/9: 处理网格参数...
步骤3/9: 创建基础DataFrame...
步骤4/9: 提取水位数据...
水位数据: 30时间步, 形状(30, 1, 30, 50)
步骤5/9: 提取污染物浓度数据...
浓度数据: 30时间步, 形状(30, 1, 30, 50)
步骤6/9: 提取其他包数据...
  - 提取CHD数据...
    CHD数据点: 31
  - 提取补给(RCH)数据...
    RCH形状: (30, 1, 30, 50)
  - 提取蒸发(EVT)数据...
    EVT数据形状: (30, 1, 30, 50)
  - 提取河流(RIV)数据...
    RIV数据点: 270
  - 提取抽水井(WEL)数据...
    WEL数据点: 10个井位置
步骤7/9: 合并时间步和层数据...
步骤8/9: 准备返回数据...
步骤9/

...

    loading package wel...
    loading package rch...
    loading package chd...
    loading package evt...
    loading package oc...
    loading package obs...
  loading model gwt6...
    loading package dis...
    loading package ic...
    loading package dsp...
    loading package mst...
    loading package ssm...
    loading package obs...
    loading package oc...
  loading exchange package gwf-gwt_exg_0...
  loading solution package gwf_con_93...
  loading solution package gwt_con_93...
找到GWT模型: gwt_con_93

开始提取模型 con_93 @ 2025-08-01 17:31:51
步骤1/9: 获取模型对象...
网格维度: nlay=1, nrow=30, ncol=50
步骤2/9: 处理网格参数...
步骤3/9: 创建基础DataFrame...
步骤4/9: 提取水位数据...
水位数据: 30时间步, 形状(30, 1, 30, 50)
步骤5/9: 提取污染物浓度数据...
浓度数据: 30时间步, 形状(30, 1, 30, 50)
步骤6/9: 提取其他包数据...
  - 提取CHD数据...
    CHD数据点: 31
  - 提取补给(RCH)数据...
    RCH形状: (30, 1, 30, 50)
  - 提取蒸发(EVT)数据...
    EVT数据形状: (30, 1, 30, 50)
  - 提取河流(RIV)数据...
    RIV数据点: 270
  - 提取抽水井(WEL)数据...
    WEL数据点: 10个井位置
步骤7/9: 合并时间步和层数据...
步骤8/9: 准备返回数据...
步骤9/

/tmp/ipykernel_480897/2557538960.py:264: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['concentration'] = cleaned_data['concentration'].clip(lower=0)


    loading package wel...
    loading package rch...
    loading package chd...
    loading package evt...
    loading package oc...
    loading package obs...
  loading model gwt6...
    loading package dis...
    loading package ic...
    loading package dsp...
    loading package mst...
    loading package ssm...
    loading package obs...
    loading package oc...
  loading exchange package gwf-gwt_exg_0...
  loading solution package gwf_con_73...
  loading solution package gwt_con_73...
找到GWT模型: gwt_con_73

开始提取模型 con_73 @ 2025-08-01 17:31:57
步骤1/9: 获取模型对象...
网格维度: nlay=1, nrow=30, ncol=50
步骤2/9: 处理网格参数...
步骤3/9: 创建基础DataFrame...
步骤4/9: 提取水位数据...
水位数据: 30时间步, 形状(30, 1, 30, 50)
步骤5/9: 提取污染物浓度数据...
浓度数据: 30时间步, 形状(30, 1, 30, 50)
步骤6/9: 提取其他包数据...
  - 提取CHD数据...
    CHD数据点: 31
  - 提取补给(RCH)数据...
    RCH形状: (30, 1, 30, 50)
  - 提取蒸发(EVT)数据...
    EVT数据形状: (30, 1, 30, 50)
  - 提取河流(RIV)数据...
    RIV数据点: 270
  - 提取抽水井(WEL)数据...
    WEL数据点: 10个井位置
步骤7/9: 合并时间步和层数据...
步骤8/9: 准备返回数据...
步骤9/

/tmp/ipykernel_480897/2557538960.py:264: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['concentration'] = cleaned_data['concentration'].clip(lower=0)


总数据量: 2664000行
包含的模型: ['con_9' 'con_34' 'con_28' 'con_27' 'con_90' 'con_78' 'con_29' 'con_1'
 'con_50' 'con_14' 'con_0' 'con_62' 'con_70' 'con_39' 'con_41' 'con_24'
 'con_66' 'con_40' 'con_53' 'con_96' 'con_71' 'con_4' 'con_97' 'con_49'
 'con_89' 'con_85' 'con_91' 'con_87' 'con_31' 'con_56' 'con_26' 'con_2'
 'con_51' 'con_79' 'con_92' 'con_57' 'con_15' 'con_95' 'con_68' 'con_82'
 'con_45' 'con_98' 'con_13' 'con_5' 'con_25' 'con_67' 'con_21' 'con_35'
 'con_76' 'con_18' 'con_64' 'con_10' 'con_88' 'con_52' 'con_47' 'con_16'
 'con_54' 'con_42' 'con_11' 'con_46' 'con_86' 'con_94' 'con_58' 'con_61'
 'con_44' 'con_74' 'con_8' 'con_6' 'con_36' 'con_32' 'con_23' 'con_19'
 'con_77' 'con_20' 'con_38' 'con_72' 'con_7' 'con_22' 'con_83' 'con_37'
 'con_30' 'con_99' 'con_3' 'con_12' 'con_80' 'con_69' 'con_17' 'con_60'
 'con_75' 'con_48' 'con_65' 'con_63' 'con_84' 'con_55' 'con_33' 'con_59'
 'con_81' 'con_43' 'con_93' 'con_73']
保存合并数据到 conc_confine.csv
